In [1]:
# 安装必要的库
# !pip install pandas plotly ipywidgets google-cloud-bigquery numpy google-cloud-bigquery-storage

# 导入所需的库
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from google.cloud import bigquery
import numpy as np
from ipywidgets import widgets, Layout
from IPython.display import display, HTML
import ipywidgets as widgets
import datetime
from plotly.subplots import make_subplots

# 设置BigQuery客户端
# 替换为您的项目ID
PROJECT_ID = "brazil-olist"
client = bigquery.Client(project=PROJECT_ID)

# 创建标题样式
title_style = """
<style>
.dashboard-title {
    font-family: 'Arial', sans-serif;
    font-size: 28px;
    font-weight: bold;
    color: #1a3d7c;
    text-align: center;
    padding: 15px;
    margin-bottom: 20px;
    background-color: #f0f7ff;
    border-radius: 10px;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
}

.last-updated {
    font-size: 14px;
    color: #666;
    text-align: right;
    margin-bottom: 20px;
}

.description {
    font-size: 16px;
    color: #333;
    margin-bottom: 20px;
    padding: 15px;
    background-color: #f9f9f9;
    border-radius: 8px;
}
</style>
"""

# 显示标题和描述
display(HTML(title_style))
display(HTML('<div class="dashboard-title">巴西各地区平均送货时间分析</div>'))
display(HTML(f'<div class="last-updated">最后更新: {datetime.datetime.now().strftime("%Y-%m-%d %H:%M")}</div>'))
display(HTML('''
<div class="description">
此仪表盘展示巴西各地区的平均送货时间分析。圆点大小表示该地区的平均送货时间（时间越长，圆点越大），颜色表示送货时间分类。
使用地图工具可以缩放和查看详细信息，筛选器可用于按州或送货时间范围筛选数据。
</div>
'''))

# 从BigQuery获取数据
@widgets.interact_manual(
    refresh=widgets.ToggleButton(
        value=False,
        description='刷新数据',
        disabled=False,
        button_style='', 
        tooltip='从BigQuery获取最新数据',
        icon='refresh'
    )
)
def fetch_delivery_data(refresh):
    """从BigQuery获取送货时间数据"""
    print("正在从BigQuery加载数据...")
    
    query = f"""
    SELECT 
        zip_code_prefix,
        avg_latitude,
        avg_longitude,
        city,
        state,
        avg_delivery_days,
        delivery_category,
        order_count
    FROM `{PROJECT_ID}.dbt_output.fct_delivery_time_by_zip`
    WHERE avg_delivery_days IS NOT NULL
      AND avg_latitude IS NOT NULL
      AND avg_longitude IS NOT NULL
      AND state IS NOT NULL
    ORDER BY avg_delivery_days DESC
    """
    
    df = client.query(query).to_dataframe()
    print(f"成功加载 {len(df)} 条记录")
    
    # 计算摘要统计
    avg_days = df['avg_delivery_days'].mean()
    min_days = df['avg_delivery_days'].min()
    max_days = df['avg_delivery_days'].max()
    
    # 显示摘要信息
    stats_html = f"""
    <div style="background-color:#f0f8ff; padding:15px; border-radius:8px; margin-bottom:20px;">
        <h3 style="color:#1a3d7c;">数据摘要</h3>
        <p><strong>覆盖地区:</strong> {len(df['zip_code_prefix'].unique())} 个邮编前缀区域</p>
        <p><strong>覆盖城市:</strong> {len(df['city'].unique())} 个城市</p>
        <p><strong>覆盖州:</strong> {len(df['state'].unique())} 个州</p>
        <p><strong>平均送货时间:</strong> {avg_days:.1f} 天</p>
        <p><strong>最短送货时间:</strong> {min_days:.1f} 天</p>
        <p><strong>最长送货时间:</strong> {max_days:.1f} 天</p>
    </div>
    """
    display(HTML(stats_html))
    
    return df

# 初始化数据
df = fetch_delivery_data(False)

# 创建筛选器
state_list = sorted(df['state'].unique().tolist())
category_list = sorted(df['delivery_category'].unique().tolist())

state_filter = widgets.SelectMultiple(
    options=state_list,
    value=state_list,
    description='选择州:',
    disabled=False,
    layout=Layout(width='300px', height='150px')
)

category_filter = widgets.SelectMultiple(
    options=category_list,
    value=category_list,
    description='送货时间:',
    disabled=False,
    layout=Layout(width='300px', height='150px')
)

min_orders = widgets.IntSlider(
    value=5,
    min=1,
    max=100,
    step=1,
    description='最小订单数:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# 创建筛选器UI
filters = widgets.HBox([
    widgets.VBox([state_filter, category_filter]),
    widgets.VBox([min_orders])
], layout=Layout(justify_content='space-around', padding='20px', border='1px solid #ddd', border_radius='8px'))

display(filters)

# 创建交互式地图
def create_map(df, states, categories, min_order_count):
    """创建交互式巴西地图"""
    # 应用筛选器
    filtered_df = df[
        (df['state'].isin(states)) & 
        (df['delivery_category'].isin(categories)) &
        (df['order_count'] >= min_order_count)
    ]
    
    if filtered_df.empty:
        print("没有符合筛选条件的数据。请调整筛选条件。")
        return go.Figure()
    
    # 设置点的大小（基于送货时间）
    min_size, max_size = 5, 40
    filtered_df['size'] = np.interp(
        filtered_df['avg_delivery_days'],
        (filtered_df['avg_delivery_days'].min(), filtered_df['avg_delivery_days'].max()),
        (min_size, max_size)
    )
    
    # 设置颜色类别
    color_sequence = px.colors.sequential.Reds
    
    # 创建地图
    fig = px.scatter_mapbox(
        filtered_df,
        lat="avg_latitude",
        lon="avg_longitude",
        size="size",
        size_max=max_size,
        color="delivery_category",
        color_discrete_sequence=color_sequence,
        hover_name="city",
        hover_data={
            "zip_code_prefix": True,
            "avg_delivery_days": ":.1f",
            "order_count": True,
            "state": True,
            "delivery_category": False,
            "size": False
        },
        zoom=3,
        height=700,
        title="巴西各地区平均送货时间"
    )
    
    # 更新地图样式和布局
    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_center={"lat": -14.2350, "lon": -51.9253},
        mapbox_zoom=3,
        margin={"r": 0, "t": 40, "l": 0, "b": 0},
        legend_title_text="送货时间分类",
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Arial"
        ),
        title=dict(
            x=0.5,
            xanchor='center',
            font=dict(size=20)
        )
    )
    
    # 设置地图范围仅限巴西
    fig.update_mapboxes(
        bounds_east=-35,
        bounds_west=-75,
        bounds_north=5,
        bounds_south=-35
    )
    
    # 自定义悬停提示
    fig.update_traces(
        hovertemplate="<b>%{hovertext}</b><br>"
                      "州: %{customdata[3]}<br>"
                      "邮编前缀: %{customdata[0]}<br>"
                      "平均送货时间: %{customdata[1]} 天<br>"
                      "订单数量: %{customdata[2]}"
    )
    
    return fig

# 创建州分析图表
def create_state_chart(df, states):
    """创建各州送货时间分析图表"""
    # 筛选数据
    state_df = df[df['state'].isin(states)]
    
    if state_df.empty:
        return go.Figure()
    
    # 按州分组计算
    state_data = state_df.groupby('state').agg(
        avg_days=('avg_delivery_days', 'mean'),
        location_count=('zip_code_prefix', 'count'),
        total_orders=('order_count', 'sum')
    ).reset_index().sort_values('avg_days', ascending=False)
    
    # 创建图表
    fig = go.Figure()
    
    # 添加条形图
    fig.add_trace(go.Bar(
        x=state_data['state'],
        y=state_data['avg_days'],
        name='平均送货天数',
        marker_color=state_data['avg_days'],
        marker_colorscale='Reds',
        hovertemplate="%{x}: %{y:.1f}天<extra></extra>"
    ))
    
    # 添加订单数量散点图（次坐标轴）
    fig.add_trace(go.Scatter(
        x=state_data['state'],
        y=state_data['total_orders'],
        name='订单数量',
        yaxis='y2',
        mode='lines+markers',
        line=dict(color='#1f77b4', width=2),
        marker=dict(size=8, color='#1f77b4'),
        hovertemplate="%{x}: %{y} 单<extra></extra>"
    ))
    
    # 更新布局
    fig.update_layout(
        title='各州平均送货时间与订单数量',
        height=400,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        yaxis=dict(
            title='平均送货天数',
            title_font=dict(color='#d62728'),
            tickfont=dict(color='#d62728'),
            range=[0, state_data['avg_days'].max() * 1.2]
        ),
        yaxis2=dict(
            title='订单数量',
            title_font=dict(color='#1f77b4'),
            tickfont=dict(color='#1f77b4'),
            anchor='x',
            overlaying='y',
            side='right',
            range=[0, state_data['total_orders'].max() * 1.2]
        ),
        margin=dict(t=60, b=80, l=60, r=60),
        hovermode="x unified"
    )
    
    return fig

# 创建时间分类图表
def create_category_chart(df, categories):
    """创建送货时间分类图表"""
    # 筛选数据
    category_df = df[df['delivery_category'].isin(categories)]
    
    if category_df.empty:
        return go.Figure()
    
    # 计算各分类的统计信息
    category_stats = category_df.groupby('delivery_category').agg(
        avg_days=('avg_delivery_days', 'mean'),
        location_count=('zip_code_prefix', 'count'),
        total_orders=('order_count', 'sum')
    ).reset_index()
    
    # 创建图表
    fig = make_subplots(rows=1, cols=2, 
                        specs=[[{"type": "pie"}, {"type": "bar"}]],
                        subplot_titles=("地区分布", "平均送货时间"))
    
    # 添加地区分布饼图
    fig.add_trace(
        go.Pie(
            labels=category_stats['delivery_category'],
            values=category_stats['location_count'],
            name='地区分布',
            hole=0.4,
            hoverinfo="label+percent+value",
            textinfo='percent',
            marker_colors=px.colors.sequential.Reds
        ),
        row=1, col=1
    )
    
    # 添加平均送货时间条形图
    fig.add_trace(
        go.Bar(
            x=category_stats['delivery_category'],
            y=category_stats['avg_days'],
            name='平均送货时间',
            marker_color=category_stats['avg_days'],
            marker_colorscale='Reds',
            hovertemplate="%{x}: %{y:.1f}天<extra></extra>"
        ),
        row=1, col=2
    )
    
    # 更新布局
    fig.update_layout(
        title_text="送货时间分类分析",
        height=400,
        showlegend=False,
        margin=dict(t=80, b=80, l=40, r=40)
    )

    # 更新y轴标题
    fig.update_yaxes(title_text="平均送货天数 (天)", row=1, col=2)    

    return fig

# 响应式更新函数
def update_dashboard(change):
    """更新仪表盘内容"""
    # 获取筛选器值
    selected_states = state_filter.value
    selected_categories = category_filter.value
    min_order_value = min_orders.value
    
    # 清除之前的输出
    output.clear_output()
    
    with output:
        # 创建地图
        map_fig = create_map(df, selected_states, selected_categories, min_order_value)
        if map_fig.data:
            display(map_fig)
        else:
            display(HTML("<p style='color:red; text-align:center;'>没有符合筛选条件的数据</p>"))
        
        # 创建州分析图表
        state_fig = create_state_chart(df, selected_states)
        if state_fig.data:
            display(state_fig)
        
        # 创建分类图表
        category_fig = create_category_chart(df, selected_categories)
        if category_fig.data:
            display(category_fig)

# 注册筛选器事件
state_filter.observe(update_dashboard, names='value')
category_filter.observe(update_dashboard, names='value')
min_orders.observe(update_dashboard, names='value')

# 创建输出区域
output = widgets.Output(layout=Layout(border='1px solid #ddd', padding='10px', border_radius='8px'))
display(output)

# 初始渲染
update_dashboard(None)

# 添加说明
display(HTML('''
<div style="margin-top:20px; padding:15px; background-color:#f9f9f9; border-radius:8px;">
    <h3>使用说明</h3>
    <ul>
        <li><strong>地图交互</strong>: 使用鼠标滚轮缩放地图，拖动平移地图，悬停查看详细信息</li>
        <li><strong>筛选器</strong>: 选择特定州或送货时间范围，设置最小订单数阈值</li>
        <li><strong>圆点大小</strong>: 圆点越大表示平均送货时间越长</li>
        <li><strong>颜色编码</strong>: 颜色越深表示送货时间越长</li>
    </ul>
    <p><strong>提示</strong>: 放大后可查看邮编前缀级别的详细信息</p>
</div>
'''))

interactive(children=(ToggleButton(value=False, description='刷新数据', icon='refresh', tooltip='从BigQuery获取最新数据')…

正在从BigQuery加载数据...
成功加载 6575 条记录


Output(layout=Layout(border_bottom='1px solid #ddd', border_left='1px solid #ddd', border_right='1px solid #dd…